# Investigate distribution of protein abundances

In [1]:
import altair as alt
import numpy as np
import oscutils
import scipy
import statsmodels.stats.multitest

alt.data_transformers.enable("data_server")

DataTransformerRegistry.enable('data_server')

In [2]:
inputs = {}

for source in ["mm", "pd"]:
    df = oscutils.\
    load_protein_table(source, "quant", clean=True).\
    drop(columns="sample").\
    set_index(["sample_type", "sample_condition", "sample_num"])
    
    inputs[source] = df

## Sample counts

See how many samples there are in each condition, so we have an idea of what might be a good cutoff for how many samples a protein has to be found in in each condition before we do a t-test.

In [3]:
def get_sample_counts(inputs_dict):
    
    sources = []
    sample_types = []
    sample_conditions = []
    counts = []
    
    df = inputs_dict["mm"] # Doesn't matter which we use, sample counts are the same for both
    for sample_type in df.index.get_level_values("sample_type").unique():
        for sample_condition in df.index.get_level_values("sample_condition").unique():
            df_sel = df.xs((sample_type, sample_condition), level=("sample_type", "sample_condition"), drop_level=False)

            sources.append(source)
            sample_types.append(sample_type)
            sample_conditions.append(sample_condition)
            counts.append(df_sel.shape[0])
    
    return pd.DataFrame({
        "sample_type": sample_types,
        "sample_condition": sample_conditions,
        "count": counts,
    })
                
get_sample_counts(inputs)

,sample_type,sample_condition,count
0,boost,healthy,3
1,boost,unhealthy,3
2,hfl1,healthy,36
3,hfl1,unhealthy,35
4,pbulk,healthy,3
5,pbulk,unhealthy,3


## Histogram of proportion of proteins with different coverages

Make a histogram showing, for each number of samples in each condition, what proportion of proteins are found in that number of samples

In [4]:
def get_protein_coverages(inputs_dict):
    
    sources = []
    sample_types = []
    sample_conditions = []
    samples_counts = []
    proteins = []
    samples_found_counts = [] 
    
    for source in inputs_dict.keys():
        df = inputs_dict[source]
        for sample_type in df.index.get_level_values("sample_type").unique():
            for sample_condition in df.index.get_level_values("sample_condition").unique():
                df_sel = df.xs((sample_type, sample_condition), level=("sample_type", "sample_condition"), drop_level=False)
                for protein in df_sel.columns:
                    
                    sources.append(source)
                    sample_types.append(sample_type)
                    sample_conditions.append(sample_condition)
                    samples_counts.append(df_sel.shape[0])
                    proteins.append(protein)
                    samples_found_counts.append(df_sel[protein].notna().sum())
    
    coverages_df = pd.DataFrame({
        "source": sources,
        "sample_type": sample_types,
        "sample_condition": sample_conditions,
        "samples_count": samples_counts,
        "protein": proteins,
        "samples_found_count": samples_found_counts,
    })
    
    coverages_df = coverages_df.assign(
        samples_found_prop=coverages_df["samples_found_count"] / coverages_df["samples_count"],
        source_type_condition=coverages_df["source"] + "_" + coverages_df["sample_type"] + "_" + coverages_df["sample_condition"],
        source_condition=coverages_df["source"] + "_" + coverages_df["sample_condition"],
    )
    
    return coverages_df
                
coverages = get_protein_coverages(inputs)

In [5]:
alt.vconcat(*[
    alt.hconcat(*[
        alt.Chart(coverages[
            (coverages["sample_type"] == sample_type) &
            (coverages["source_condition"] == source_condition)
        ]).transform_joinaggregate(
            total="count(*)",
        ).transform_calculate(
            pct="1 / datum.total",
        ).mark_bar().encode(
            x=alt.X(
                "samples_found_count",
                bin=alt.Bin(
                    extent=[0, 37] if sample_type == "hfl1" else [0, 4], 
                    step=1
                ),
                title=[
                    "Number of samples each protein is found in",
                    f"{sample_type}: {source_condition}",
                ],
            ),
            y=alt.Y(
                "sum(pct):Q",
                title="Percentage of all proteins",
                axis=alt.Axis(
                    format="%",
                ),
            ),
            color="sample_condition",
        )
        
        for source_condition in coverages["source_condition"].unique()
    ]).resolve_scale(
        x="independent",
        y="shared",
    )
    
    for sample_type in coverages["sample_type"].unique()
])

alt.VConcatChart(...)

## Distributions of abundances for each protein

For each protein in each sample, show what the distribution of abundances looks like.

In [6]:
def individual_distributions_plot(df):
    
    df = df.sample(
        n=25,
        replace=False,
        random_state=0,
        axis=1,
        ignore_index=False,
    )
    
    df = pd.melt(
        frame=df,
        var_name="protein",
        value_name="abundance",
        ignore_index=False,
    ).\
    reset_index(drop=False).\
    sort_values(by=["sample_type", "protein"]).\
    dropna(
        axis=0,
        how="all",
        subset="abundance",
    )
    
    return alt.vconcat(*[
        alt.hconcat(*[
            alt.hconcat(*[
            
                alt.Chart(df[
                    (df["sample_type"] == sample_type) &
                    (df["sample_condition"] == sample_condition) &
                    (df["protein"] == protein)
                ]).mark_bar().encode(
                    x=alt.X(
                        "abundance:Q",
                        title=f"{sample_condition} - {protein}",
                        bin=alt.Bin(
                        ),
                    ),
                    y="count()",
                    color="sample_type:N"
                )

                for sample_condition in df["sample_condition"].unique()
            ])
            for protein in df["protein"].unique()
        ]) 
        for sample_type in df["sample_type"].unique()
    ])

In [7]:
individual_distributions_plot(inputs["mm"])

alt.VConcatChart(...)

In [8]:
individual_distributions_plot(inputs["pd"])

alt.VConcatChart(...)

## Distribution of normalized abundances across all proteins

To see overall.

In [9]:
def overall_distribution_plot(inputs_dict, log2):
    
    normalized = pd.DataFrame()
    
    for source in inputs_dict.keys():
        df = inputs_dict[source]
        for sample_type in df.index.get_level_values("sample_type").unique():
            for sample_condition in df.index.get_level_values("sample_condition").unique():

                df_sel = df.xs((sample_type, sample_condition), level=("sample_type", "sample_condition"), drop_level=False)

                means = df_sel.mean(axis=0)
                maxes = df_sel.max(axis=0)
                mins = df_sel.min(axis=0)

                df_sel = df_sel.melt(var_name="protein", value_name="abundance", ignore_index=False)

                stats = pd.DataFrame({
                    "source": source,
                    "mean": means,
                    "max": maxes,
                    "min": mins,
                })

                df_sel = df_sel.merge(
                    right=stats,
                    left_on="protein", 
                    right_index=True,
                    how="outer",
                )

                df_sel = df_sel.assign(
                    norm_abundance=(df_sel["abundance"] - df_sel["mean"]) / (df_sel["max"] - df_sel["min"]),
                )
                df_sel = df_sel.reset_index(drop=False)[["source", "sample_type", "protein", "abundance", "norm_abundance"]]

                normalized = pd.concat([normalized, df_sel], axis=0)

    normalized = normalized.dropna(how="any")
    normalized.loc[np.isinf(normalized["norm_abundance"]), "norm_abundance"] = 0
    
    if log2:
        normalized = normalized.assign(log2_norm=np.log2(normalized["norm_abundance"] + 1))
    
    chart = alt.vconcat(*[
        alt.Chart(normalized[normalized["source"] == source]).mark_bar().encode(
            x=alt.X(
                "log2_norm" if log2 else "norm_abundance",
                bin=alt.Bin(
                    step=0.05,
                ),
            ),
            y="count()",
            column="sample_type",
        ).properties(
            title=source,
        )
        
        for source in inputs_dict.keys()
    ]).resolve_scale(
        x="shared",
        y="shared",
    )
    
    return chart

In [10]:
overall_distribution_plot(inputs, log2=False)

alt.VConcatChart(...)

In [11]:
overall_distribution_plot(inputs, log2=True)

alt.VConcatChart(...)

In [12]:
# Plan: Test for normality, try with log transform if fail, MWU if still fail; check variance if succeed, Welch if different variance
# Confirm PD data already normalized

## Shapro-Wilk test for normality

In [13]:
import warnings
warnings.simplefilter("always")

def shapiro_wilk(inputs_dict):
    
    results = {}
    
    for source in inputs_dict.keys():
        df = inputs_dict[source]

        sample_types = []
        sample_conditions = []
        prots = []
        pvals = []

        for sample_type in df.index.get_level_values("sample_type").unique():
            for sample_condition in df.index.get_level_values("sample_condition").unique():
                df_sel = df.xs((sample_type, sample_condition), level=("sample_type", "sample_condition"), drop_level=False)
                for prot in df_sel.columns:
                    
                    abundances = df_sel[prot].dropna()

                    if sample_type in ["boost", "pbulk"]:
                        min_count = 3
                    else:
                        min_coun = 15

                    if len(abundances) >= min_count:
                        stat, p = scipy.stats.shapiro(abundances)
                    else:
                        p = np.nan

                    sample_types.append(sample_type)
                    sample_conditions.append(sample_condition)
                    prots.append(prot)
                    pvals.append(p)
                    
        raw_pvals = pd.DataFrame({
            "sample_type": sample_types,
            "sample_condition": sample_conditions,
            "protein": prots,
            "p_uncorrected": pvals,
        })

        pvals = raw_pvals[raw_pvals["p_uncorrected"].notna()]

        # Correct the p values
        reject, pvals_corrected, alphacSidak, alphacBonf = statsmodels.stats.multitest.multipletests(
            pvals=pvals["p_uncorrected"].dropna(),
            alpha=0.05,
            method="fdr_bh",
        )

        pvals = pvals.assign(p_corrected=pvals_corrected) 
        
        results[source] = pvals

    return results

sw = shapiro_wilk(inputs)

/home/caleb/.pyenv/versions/3.10.11/lib/python3.10/site-packages/scipy/stats/_morestats.py:1813: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


In [14]:
sources = []
sample_types = []
sample_conditions = []
counts = []
props_uncorrected_sig = []
props_corrected_sig = []

for source in sw.keys():
    df = sw[source]
    for sample_type in df["sample_type"].unique():
        for sample_condition in df["sample_condition"].unique():
            
            df_sel = df[(df["sample_type"] == sample_type) & (df["sample_condition"] == sample_condition)]
            
            sources.append(source)
            sample_types.append(sample_type)
            sample_conditions.append(sample_condition)
            
            if df_sel.shape[0] > 0:
                props_uncorrected_sig.append((df_sel["p_uncorrected"] <= 0.05).sum() / df.shape[0])
                props_corrected_sig.append((df_sel["p_corrected"] <= 0.05).sum() / df.shape[0])
            else:
                props_uncorrected_sig.append(np.nan)
                props_corrected_sig.append(np.nan)
            
pd.DataFrame({
    "source": sources,
    "sample_type": sample_types,
    "sample_condition": sample_conditions,
    "prop_uncorrected_sig": props_uncorrected_sig,
    "prop_corrected_sig": props_corrected_sig,
})

,source,sample_type,sample_condition,prop_uncorrected_sig,prop_corrected_sig
0,mm,boost,healthy,0.009020,0.002197
1,mm,boost,unhealthy,0.007633,0.002313
2,mm,hfl1,healthy,0.135885,0.124899
3,mm,hfl1,unhealthy,0.158668,0.151151
4,mm,pbulk,healthy,0.008442,0.002313
5,mm,pbulk,unhealthy,0.006823,0.002197
6,pd,boost,healthy,0.012839,0.002842
7,pd,boost,unhealthy,0.009311,0.001568
8,pd,hfl1,healthy,0.106439,0.070862
9,pd,hfl1,unhealthy,0.129668,0.093012


In [15]:
sw["pd"][sw["pd"]["protein"] == "O75323"]

,sample_type,sample_condition,protein,p_uncorrected,p_corrected
995,boost,healthy,O75323,0.357152,0.612661
3893,boost,unhealthy,O75323,0.753336,0.885786
6791,hfl1,healthy,O75323,0.000109,0.001582
9689,hfl1,unhealthy,O75323,0.025394,0.114260
12587,pbulk,healthy,O75323,0.019047,0.092893
